In [44]:
from nba_api.stats.endpoints import playergamelog, boxscoreadvancedv2
import pandas as pd
import numpy as np
import time
import random
import sys
import sqlite3
from dotenv import load_dotenv
import os
import ace_tools_open as tools

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

load_dotenv()

True

In [34]:
def database_connect():

    DB_PATH = os.path.join(os.getenv("DB_PATH"), "nba_data.db")
    print(DB_PATH)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    return conn, cursor

___________
## Gamelog With Advanced Stats

In [8]:
class PlayerProfile:
    def __init__(self, name, player_id, seasons):
        self.name = name
        self.id = player_id
        self.seasons = seasons
        self.gamelog = self.collect_stats(player_id, seasons)
        self.advanced_stats = self

    def __str__(self):
        return self.name  # This controls what gets printed when using print(object)

    def __repr__(self):
        return f"PlayerProfile(name={self.name!r})"
    
    def collect_stats(self, player_id, seasons):
        gamelog = self.collect_gamelog(player_id, seasons)
        gamelog = self.collect_adv_stats(gamelog, player_id)

        return gamelog

    def collect_gamelog(self, player_id, seasons):
        gamelog = pd.DataFrame()
        for season in seasons:
            data = playergamelog.PlayerGameLog(player_id=player_id, season=season)
            data = data.get_dict()
            if gamelog.empty:
                cols = data['resultSets'][0]['headers']
                gamelog = pd.DataFrame(columns=cols)
            observations = data['resultSets'][0]['rowSet']
            gamelog = pd.concat([gamelog, pd.DataFrame(observations, columns=gamelog.columns)], ignore_index=True)

        return gamelog
    
    def collect_adv_stats(self, gamelog, player_id):
        '''manage advanced stats'''
        categories = ['E_USG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'PACE_PER40', 'TS_PCT', 'AST_PCT']

        # Ensure gamelog has the columns for advanced stats
        for col in categories:
            gamelog[col] = None  # Initialize empty columns
        
        # Iterate over each game entry
        for idx, row in gamelog.iterrows():
            game_id = row['Game_ID']
            
            try:
                # Print dynamic status update (overwriting the previous game_id)
                sys.stdout.write(f"\rProcessing game: {game_id}... ")
                sys.stdout.flush()

                # Introduce a random delay before making the API request
                sleep_time = random.uniform(1,2)  # Sleep between 1 to 5 seconds
                time.sleep(sleep_time)
                stats = self.adv_stats_by_game(game_id, player_id)  # Retrieve stats
                for col in categories:
                    gamelog.at[idx, col] = stats[col]  # Update DataFrame
            except AssertionError as e:
                print(f"Skipping game {game_id} due to error: {e}")
            except Exception as e:
                print(f"Unexpected error retrieving stats for game {game_id}: {e}")

        return gamelog  # Return the updated DataFrame
        


    def adv_stats_by_game(self, game_id, player_id):
        categories = ['E_USG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'PACE_PER40', 'TS_PCT', 'AST_PCT']
        result = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=str(game_id))
        result = result.get_dict()
        data = pd.DataFrame(
            data=result['resultSets'][0]['rowSet'],
            columns=result['resultSets'][0]['headers'])
        pl = data[data['PLAYER_ID'] == int(player_id)]
        assert len(pl) == 1, "Too many rows returned for game/player advanced stats pull."
        return pl[categories].iloc[0].to_dict()






In [9]:
SEASONS = [
    '2024-25',
    '2023-24',
    '2022-23',
    '2021-22',
    '2020-21'
]
PLAYER_ID = '1627759'
NAME = 'Jalen Brown'
player = PlayerProfile(
    name=NAME,
    player_id=PLAYER_ID,
    seasons=SEASONS
)
print(player)

/var/folders/fm/7zh52wdj2mx7v17vqtcy7ggc0000gn/T/ipykernel_64949/4164399261.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamelog = pd.concat([gamelog, pd.DataFrame(observations, columns=gamelog.columns)], ignore_index=True)


Processing game: 0022101078... Unexpected error retrieving stats for game 0022101078: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Processing game: 0022101042... Unexpected error retrieving stats for game 0022101042: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Processing game: 0022000003... Jalen Brown


In [11]:
### Cleanup Code for missing values
missed_game = '0022101042'
categories = ['E_USG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'PACE_PER40', 'TS_PCT', 'AST_PCT']
data = player.adv_stats_by_game(game_id=missed_game, player_id=player.id)
for cat, val in data.items():
    player.gamelog.loc[(player.gamelog['Game_ID'] == missed_game), cat] = val


In [40]:
len(player.gamelog.columns)

37

In [13]:
def extract_teams(row):
    parts = row["MATCHUP"].split(" ")
    if "vs." in row["MATCHUP"]:
        home_team, away_team = parts[0], parts[2]
    elif "@" in row["MATCHUP"]:
        away_team, home_team = parts[0], parts[2]
    return pd.Series({"home_team_abbr": home_team, "away_team_abbr": away_team})

player.gamelog[["home_team_abbr", "away_team_abbr"]] = player.gamelog.apply(extract_teams, axis=1)

In [38]:
# Merge with team IDs using the Teams table
conn, cursor = database_connect()
teams_df = pd.read_sql("SELECT * FROM Teams;", conn)

# Create a mapping of abbreviations to team IDs
team_abbrev_to_id = dict(zip(teams_df["abbreviation"], teams_df["id"]))

# Map team abbreviations to team IDs
player.gamelog["home_team_id"] = player.gamelog["home_team_abbr"].map(team_abbrev_to_id)
player.gamelog["away_team_id"] = player.gamelog["away_team_abbr"].map(team_abbrev_to_id)

conn.close()

/Users/matth/projects/player-prop-simulation/data/nba_data.db


In [42]:
# Connect to SQLite
conn, cursor = database_connect()

# Step 1: Auto-generate SQL table creation query
columns = ", ".join([f"{col} REAL" if player.gamelog[col].dtype in ['float64', 'int64'] else f"{col} TEXT" for col in player.gamelog.columns])

create_table_query = f"""
CREATE TABLE IF NOT EXISTS PlayerStats (
    {columns},
    PRIMARY KEY (game_id, player_id)  -- Enforce uniqueness on player-game pairs
);
"""
cursor.execute(create_table_query)
conn.commit()

conn.close()

/Users/matth/projects/player-prop-simulation/data/nba_data.db


In [45]:
conn, cursor = database_connect()

# Step 2: Bulk Insert Data Efficiently
player.gamelog.to_sql("PlayerStats", conn, if_exists="append", index=False)

print("✅ PlayerStats table created and data inserted successfully!")

# Display the table for verification
query = "SELECT * FROM PlayerStats LIMIT 5;"
df = pd.read_sql(query, conn)

tools.display_dataframe_to_user(name="PlayerStats Table Preview", dataframe=df)

# Close connection
conn.close()

/Users/matth/projects/player-prop-simulation/data/nba_data.db
✅ PlayerStats table created and data inserted successfully!
PlayerStats Table Preview


_________________________________

In [53]:
from nba_api.stats.endpoints import teamestimatedmetrics

season_map = {
    '2024-25': '22024',
    '2023-24': '22023',
    '2022-23': '22022',
    '2021-22': '22021',
    '2020-21': '22020'
}

full_table = pd.DataFrame()

for season_key, season_abv in season_map.items():
    ret = teamestimatedmetrics.TeamEstimatedMetrics(season=season_key)
    team_stats = ret.get_dict()
    data = pd.DataFrame(
        data=team_stats['resultSet']['rowSet'],
        columns=team_stats['resultSet']['headers'])

    categories = ['TEAM_NAME','TEAM_ID','E_OFF_RATING','E_DEF_RATING', 'E_PACE']
    data = data[categories]
    data[['SEASON_ID']] = season_abv
    if full_table.empty:
        full_table = pd.DataFrame(columns=list(data.columns))
    full_table = pd.concat([full_table, data], ignore_index=True)

full_table


/var/folders/fm/7zh52wdj2mx7v17vqtcy7ggc0000gn/T/ipykernel_64949/2254769507.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_table = pd.concat([full_table, data], ignore_index=True)


,TEAM_NAME,TEAM_ID,E_OFF_RATING,E_DEF_RATING,E_PACE,SEASON_ID
0,Memphis Grizzlies,1610612763,114.6,109.4,106.9,22024
1,Atlanta Hawks,1610612737,110.1,112.4,105.7,22024
2,Chicago Bulls,1610612741,110.5,113.7,105.4,22024
3,Washington Wizards,1610612764,103.6,115.9,104.0,22024
4,Denver Nuggets,1610612743,117.2,111.9,102.7,22024
5,Cleveland Cavaliers,1610612739,120.4,108.2,102.6,22024
6,Oklahoma City Thunder,1610612760,116.7,104.0,102.5,22024
7,Detroit Pistons,1610612765,111.2,109.9,102.4,22024
8,Utah Jazz,1610612762,108.9,116.7,102.3,22024
9,Milwaukee Bucks,1610612749,112.4,110.0,102.1,22024


In [58]:
# Connect to SQLite
conn, cursor = database_connect()

# Step 1: Auto-generate SQL table creation query
columns = ", ".join([f"{col} REAL" if full_table[col].dtype in ['float64', 'int64'] else f"{col} TEXT" for col in full_table.columns])

create_table_query = f"""
CREATE TABLE IF NOT EXISTS SeasonStats (
    {columns},
    PRIMARY KEY (TEAM_ID, SEASON_ID)  -- Enforce uniqueness on player-game pairs
);
"""
cursor.execute(create_table_query)
conn.commit()

conn.close()

/Users/matth/projects/player-prop-simulation/data/nba_data.db


In [60]:
conn, cursor = database_connect()

# Step 2: Bulk Insert Data Efficiently
full_table.to_sql("SeasonStats", conn, if_exists="append", index=False)

print("✅ SeasonStats data inserted successfully!")

# Display the table for verification
query = "SELECT * FROM SeasonStats LIMIT 5;"
df = pd.read_sql(query, conn)

tools.display_dataframe_to_user(name="SeasonStats Table Preview", dataframe=df)

# Close connection
conn.close()

/Users/matth/projects/player-prop-simulation/data/nba_data.db
✅ SeasonStats data inserted successfully!
SeasonStats Table Preview


In [62]:
# Display the table for verification
conn, cursor = database_connect()
query = "SELECT COUNT(*) AS TOTAL_ROWS FROM SeasonStats;"
df = pd.read_sql(query, conn)
conn.close()

df

/Users/matth/projects/player-prop-simulation/data/nba_data.db


,TOTAL_ROWS
0,150


_____________
